# 🧠 AI Lactate Advisor – Final Training Notebook (Merged & Clean)

This notebook trains and versions two models:

1. **Lactate Model** – predicts blood lactate (mmol/L) from time-series features  
2. **Recovery Model** – predicts recovery/readiness score from biomarker data  

It is designed to be consistent with the **Streamlit app**, saving models into `models/`
and appending results to `models/training_log.csv`.


## 📗 Cell 1 — Imports & Paths

In [ ]:
# =============================================================
# 📘 Cell 1 — Imports & Global Config
# =============================================================

import os
import numpy as np
import pandas as pd
from datetime import datetime
import joblib

from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

from github import Github

# model_utils functions
from model_utils import (
    prepare_features,
    add_hr_slopes,
    add_rolling_features,
)

DATA_DIR = "data"
MODELS_DIR = "models"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

# Model output paths (latest versions)
LACTATE_MODEL_PATH = os.path.join(MODELS_DIR, "lactate_lightgbm_model.joblib")
RECOVERY_MODEL_PATH = os.path.join(MODELS_DIR, "recovery_lightgbm_model.joblib")


## 📗 Cell 2 — Load & Merge New Data Automatically

In [ ]:
# =============================================================
# 📗 Cell 2 — Auto-detect NEW CSVs and merge into master dataset
# =============================================================

MASTER_DATASET = os.path.join(DATA_DIR, "athlete_training_dataset_with_biomarkers.csv")

# Find all CSVs in /data
csv_files = [f for f in os.listdir(DATA_DIR) if f.endswith(".csv")]

if not csv_files:
    raise FileNotFoundError("❌ No CSV files found in /data folder.")

# Newest file
latest_file = max(csv_files, key=lambda f: os.path.getmtime(os.path.join(DATA_DIR, f)))
latest_path = os.path.join(DATA_DIR, latest_file)

print("📄 Latest detected dataset:", latest_file)

# Load master dataset
if os.path.exists(MASTER_DATASET):
    df_master = pd.read_csv(MASTER_DATASET)
    master_mtime = os.path.getmtime(MASTER_DATASET)
else:
    df_master = pd.DataFrame()
    master_mtime = 0

# Compare timestamps
latest_mtime = os.path.getmtime(latest_path)

if latest_mtime > master_mtime:
    print("📦 New dataset detected — merging into master file.")
    df_new = pd.read_csv(latest_path)

    if not df_master.empty:
        df_merged = pd.concat([df_master, df_new], ignore_index=True).drop_duplicates()
    else:
        df_merged = df_new

    df_merged.to_csv(MASTER_DATASET, index=False)
    df = df_merged
    retrain_required = True
else:
    print("✅ No new data — loading current master file.")
    df = pd.read_csv(MASTER_DATASET)
    retrain_required = False

print("📊 Final dataset shape:", df.shape)


##  📗 Cell 3 — Feature Preparation (Lactate + Recovery)

In [ ]:
# =============================================================
# 📗 Cell 3 — Feature Engineering for Lactate & Recovery Models
# =============================================================

assert 'lactate' in df.columns, "❌ Missing 'lactate' column in merged dataset!"
assert 'recovery_score' in df.columns, "❌ Missing 'recovery_score' column!"

# Add engineered features
df_feat = df.copy()
df_feat = add_hr_slopes(df_feat)
df_feat = add_rolling_features(df_feat, window=30)

# -------- Lactate Data --------
X_lac = prepare_features(df_feat)
y_lac = df_feat['lactate']

print("Lactate features:", X_lac.shape)

# -------- Recovery Data --------
recovery_cols = ['heart_rate', 'power', 'cadence', 'hr_slope_time']

X_rec = df_feat[recovery_cols].copy()
y_rec = df_feat['recovery_score']

print("Recovery features:", X_rec.shape)


## 📗 Cell 4 — Model Training (Lactate + Recovery)

In [ ]:
# =============================================================
# 📗 Cell 4 — Train/Test Split + Model Training
# =============================================================

def train_lightgbm_model(X, y, model_name):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = LGBMRegressor(
        n_estimators=400,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    print(f"🚀 Training {model_name} ...")
    model.fit(X_train, y_train)

    # Embedding feature schema
    model.feature_names_in_ = list(X.columns)

    # Validation
    y_pred = model.predict(X_val)
    r2 = r2_score(y_val, y_pred)
    mae = mean_absolute_error(y_val, y_pred)

    print(f"📈 {model_name} — R2={r2:.3f}, MAE={mae:.3f}")

    return model, r2, mae

# Train Lactate
lactate_model, r2_lac, mae_lac = train_lightgbm_model(X_lac, y_lac, "lactate")

# Train Recovery
recovery_model, r2_rec, mae_rec = train_lightgbm_model(X_rec, y_rec, "recovery")


## 📗 Cell 5 — Save Models + Version + Log Metrics

In [ ]:
# =============================================================
# 📗 Cell 5 — Save Models + Versioning + Log Training Metrics
# =============================================================

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save latest
joblib.dump(lactate_model, LACTATE_MODEL_PATH)
joblib.dump(recovery_model, RECOVERY_MODEL_PATH)

# Save versioned
joblib.dump(lactate_model, f"{MODELS_DIR}/lactate_lightgbm_{timestamp}.joblib")
joblib.dump(recovery_model, f"{MODELS_DIR}/recovery_lightgbm_{timestamp}.joblib")

print("💾 Models saved.")

# ---- Append training log ----
log_path = os.path.join(MODELS_DIR, "training_log.csv")

log_entry = pd.DataFrame([{
    "timestamp": timestamp,
    "r2_lactate": r2_lac,
    "mae_lactate": mae_lac,
    "r2_recovery": r2_rec,
    "mae_recovery": mae_rec,
    "rows": len(df)
}])

if os.path.exists(log_path):
    log_entry.to_csv(log_path, mode='a', header=False, index=False)
else:
    log_entry.to_csv(log_path, index=False)

print("📘 Training metrics logged to training_log.csv")
